# Assignment 03

**Specifications and requirements** for each assignment include compliance with the [Programmer's Pact](../housekeeping/ProgrammerPact_Python_2026.pdf). In addition:

- you may **not** use the `in` operator for lists. (Perfectly fine to use it in a for loop, e.g `for x in range(something)`).
- you may **not** import any modules (ie no `import` statement allowed).
- no sets or dictionaries may be used.

This assignment has 3 problems:

- Avoid duplicate entries in a `Cast` object
- Remove a `Character` from a `Cast` object
- Comparisons

## What to submit

A single Python file called `week03.py` with the code for the following problems.


---

## Codebase

For this assignment, you will work with the following classes. Specifically, you'll add two new methods to class `Cast`.


In [ ]:
class Character:
    """A class to represent a character in a show."""

    def __init__(self, first_name: str, last_name: str, role: str):
        """Initialize a Character object with the specified first name,
        last name, and role."""
        self.__first_name = first_name
        self.__last_name = last_name
        self.__role = role

    def __str__(self) -> str:
        """Returns a string representation of the Character object. Specifically,
        the string consists of the first letter of the first name, last name,
        and role concatenated together.
        """
        return (
            ("" if self.__first_name == "" else self.__first_name[0])
            + ("" if self.__last_name == "" else self.__last_name[0])
            + ("" if self.__role == "" else self.__role[0])
        )

    def __repr__(self) -> str:
        """Returns a string representation of the Character object for debugging"""
        return self.__str__()

    def get_first_name(self) -> str:
        """Accessor for the first name of the character."""
        return self.__first_name

    def get_last_name(self) -> str:
        """Accessor for the last name of the character."""
        return self.__last_name

    def get_role(self) -> str:
        """Accessor for the role of the character."""
        return self.__role

In [ ]:
class Cast:
    """A class to represent the cast of a show, consisting of multiple
    Character objects."""

    def __init__(self, title: str):
        # The title of the show being represented
        self.__title = title
        # A list of Character objects
        self.__underlying = []

    def __len__(self) -> int:
        """Return the number of characters in the show. This allows
        the use of len() on Cast objects."""
        return len(self.__underlying)

    def __bool__(self) -> bool:
        """Return True if there is at least one character in the
        show, False otherwise. This allows the use of bool() on
        Cast objects."""
        return len(self.__underlying) > 0

    def add_character(self, first_name: str, last_name: str, role: str) -> None:
        """Add a new character to the show."""
        # First create a new Character object, then append it to the
        # object's underlying list. The following two steps can be
        # done in one step, but for illustration purposes they are
        # shown separately here.
        new_character = Character(first_name, last_name, role)
        self.__underlying.append(new_character)

    def __contains_by(self, target: str, getter) -> bool:
        """Return True if any character's getter() equals target.
        Here's something wonderful about Python: we can pass functions
        as arguments to other functions. In this case, we can pass in
        any of the getter methods defined in the Character class
        (get_first_name, get_last_name, or get_role) and use that
        to compare against the target string. This allows us to
        avoid duplicating code for each of the three "contains" methods.

        This method is private (indicated by the leading double underscore)
        because it is only intended to be used internally by the other
        "contains" methods."""
        # Prepare to iterate through the underlying list
        i = 0
        # Assume we wont find what we are looking for
        found = False
        # Iterate through the list until we either find a match
        # or run out of items to check
        while i < len(self.__underlying) and not found:
            # Check if the current character's getter() matches target
            # Note that we call the getter function passed in as an argument
            # rather than trying to access it as an attribute of the Character
            # object. If we get a match, variable found becomes True and
            # the loop will exit.
            found = getter(self.__underlying[i]) == target
            # Move to the next character in the list
            i += 1
        # Done searching, return whether we found a match
        return found

    def contains_first_name(self, first_name: str) -> bool:
        """Return True if any character has the specified first name.
        This is a public method that uses the private __contains_by()
        method to perform the actual search."""
        return self.__contains_by(first_name, Character.get_first_name)

    def contains_last_name(self, last_name: str) -> bool:
        """Return True if any character has the specified last name.
        This is a public method that uses the private __contains_by()
        method to perform the actual search."""
        return self.__contains_by(last_name, Character.get_last_name)

    def contains_role(self, role: str) -> bool:
        """Return True if any character has the specified role.
        This is a public method that uses the private __contains_by()
        method to perform the actual search."""
        return self.__contains_by(role, Character.get_role)

    # Constant string template for the report header - used in report()
    # In compliance with the "no magic value" principle
    _REPORT_HEADER = 'There are {} characters in your data about "{}"'

    def report(self) -> str:
        """Generate a nicely formatted report of all characters in the show."""
        output = self._REPORT_HEADER.format(len(self.__underlying), self.__title)
        for character in self.__underlying:
            output += f"\n\t{character.get_first_name()} {character.get_last_name()} - {character.get_role()}"
        return output

The code above uses a very Pythonic way to implement the `contains` methods. Python treats functions as _first class values._ This is a bit of an advanced concept for now, so take `__contain_by` and the `contains_first_name`, `contains_last_name`, and `contains_role` as given and guaranteed to work. If you'd like to learn more about the topic, you can start with the [Wikipedia entry for first-class values](https://en.wikipedia.org/wiki/First-class_function). For more, plan to take Konstantin Laufer's _Programming Languages_ class -- you can thank me later.

Another unusual bit of code above is the use of `str.format` in method `Cast.report`. Again, treated as a black box guaranteed to work.

Method `Character.__str__` uses the [_ternary operator_](https://docs.python.org/3/reference/expressions.html#conditional-expressions) (also known as a _conditional expression)._ It's a single-line version of an `if`/`else` statement and very, very useful. For example, the code


In [ ]:
N = 5
result = "Odd" if N % 2 else "Even"
print(result)

is equivalent to the code:


In [ ]:
N = 5
if N % 2 == 1:
    result = "Odd"
else:
    result = "Even"
print(result)

---

### Problem 1: Avoid duplicate entries

Improve class `Cast` by writing a method

```python
add_unique(
    self,
    first_name: str,
    last_name: str,
    role: str
) -> bool:
```

that adds a `Character` to the `Cast` object only if there is no other object with the same first name, last name, and role description. The method returns `True` if the addition was succesful and `False` otherwise.


---

### Problem 2: Remove a `Character` from `Cast`

Improve class `Cast` by writing a method

```python
remove(
    self,
    first_name: str,
    last_name: str,
    role: str
) -> Character | None
```

that removes and returns the first `Character` object that matches the specified first name and last name and role. If no such object exists, the method shall return `None`.



---

### Problem 3: Comparisons

Consider two `Character` objects:

```python 
a = Character("Hans", "Solo", "Adventurer")
b = Character("Luke", "Skywalker", "Jedi Knight")
```
(Alternatively, imagine two characters from your favorite book, movie, series, or game).

In 200-300 words, discuss how to compare these two characters. Specificically, how can you decide which is the best of the two? 

Include this discussion as a *docstring* block at the end of your `week03.py` file, for example

```python
"""
Comparison discussion
Hans Solo is objectively, mathematically, spiritually, and snack-wise the best pilot 
ever, not because of facts (which are optional) but because the universe itself tends 
to scoot its furniture out of the way when he flies by, muttering apologies. His ship 
obeys him the way shoelaces obey gravity: reluctantly but inevitably, even when tied 
wrong.
"""
```